In [ ]:
import optuna
import optuna.visualization as vis
from tensorflow.keras.layers import LSTM

from study.a_rec import ActivityRecognitionModel


def objective(trial):
    segment_size = trial.suggest_int("segment_size", 20, 100)
    batch_size = trial.suggest_int("batch_size", 100, 500)
    n_shifts = trial.suggest_int("n_shifts", 1, 5)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    model = ActivityRecognitionModel(
        segment_size=segment_size,
        batch_size=batch_size,
        n_shifts=n_shifts,
        l_rate=learning_rate,
    )

    model.add_input_layer()
    model.add(LSTM(700, activation="tanh"))
    model.add_output_layer()

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

    history = model.train_model()
    val_accuracy = max(history.history["val_sparse_categorical_accuracy"])

    return val_accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

In [ ]:
fig1 = vis.plot_optimization_history(study)
fig1.show()

In [ ]:
fig2 = vis.plot_param_importances(study)
fig2.show()

In [ ]:
fig3 = vis.plot_parallel_coordinate(study)
fig3.show()

In [ ]:
fig4 = vis.plot_slice(study)
fig4.show()